In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm.notebook import tqdm
import snscrape.modules.twitter as snstwitter

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import string 
import re 
import textblob
from textblob import TextBlob

import wordcloud
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer

lemmatizer = WordNetLemmatizer ()

from PIL import Image
import warnings
%matplotlib inline

# Read and perform EDA

In [2]:
df = pd.read_csv("Ukrainewar.csv")
df.head()

,Date,ID,Source,Content,Username,Location,Verified,Likes,Retweets,Place
0,2023-02-22 07:10:16+00:00,1.628290e+18,Twitter for Android,#ZelenskyWarCriminal #RussiaUkraineWar,manuel_moschini,"Florence, Italy.",False,0,0,NaN
1,2023-02-22 07:08:47+00:00,1.628290e+18,Twitter for Android,#Russia 🇷🇺 / #Ukraine 🇺🇦\n\n‼️Air raid sirens ...,_Nex3_,Italia,False,0,0,NaN
2,2023-02-22 07:07:27+00:00,1.628290e+18,Twitter Web App,"নিষেধাজ্ঞার অধীনে থাকা জাহাজ প্রত্যাখ্যান, বাং...",tbsnewsdotnet,"Dhaka, Bangladesh",False,0,0,NaN
3,2023-02-22 07:07:25+00:00,1.628290e+18,Twitter Web App,West media are not talking about President #Pu...,segunajayiemma,"Lagos,Nigeria",False,1,0,NaN
4,2023-02-22 07:07:00+00:00,1.628290e+18,Twitter for iPhone,#Russia #Ukraine #RussiaUkraineWar President D...,FalcDanny,NaN,False,0,0,NaN


In [3]:
df.drop(['ID', 'Place'], axis=1)

,Date,Source,Content,Username,Location,Verified,Likes,Retweets
0,2023-02-22 07:10:16+00:00,Twitter for Android,#ZelenskyWarCriminal #RussiaUkraineWar,manuel_moschini,"Florence, Italy.",False,0,0
1,2023-02-22 07:08:47+00:00,Twitter for Android,#Russia 🇷🇺 / #Ukraine 🇺🇦\n\n‼️Air raid sirens ...,_Nex3_,Italia,False,0,0
2,2023-02-22 07:07:27+00:00,Twitter Web App,"নিষেধাজ্ঞার অধীনে থাকা জাহাজ প্রত্যাখ্যান, বাং...",tbsnewsdotnet,"Dhaka, Bangladesh",False,0,0
3,2023-02-22 07:07:25+00:00,Twitter Web App,West media are not talking about President #Pu...,segunajayiemma,"Lagos,Nigeria",False,1,0
4,2023-02-22 07:07:00+00:00,Twitter for iPhone,#Russia #Ukraine #RussiaUkraineWar President D...,FalcDanny,NaN,False,0,0
...,...,...,...,...,...,...,...,...
9996,2023-02-05 10:30:46+00:00,Twitter Web App,"2/2 El diputado en el Parlamento ucraniano, Ol...",LevNikolay,La Habana,False,0,0
9997,2023-02-05 10:29:28+00:00,Twitter Web App,"1/2 Que todo esto se veía venir, lo sabían tod...",LevNikolay,La Habana,False,1,1
9998,2023-02-05 10:27:12+00:00,Twitter for Android,The prosecutor's office publishes footage of t...,GYI_ua,"Kropyvnytskyi 🇺🇦, Ukraine",False,0,0
9999,2023-02-05 10:25:07+00:00,Twitter for Android,"😎🇺🇦 A secret Ukrainian battalion ""Bratstvo"" is...",GYI_ua,"Kropyvnytskyi 🇺🇦, Ukraine",False,0,1


In [4]:
df1 = pd.read_csv("Food.csv")
df1.head()

,Date,ID,Source,Content,Username,Location,Verified,Likes,Retweets,Place
0,2023-02-22 05:09:32+00:00,1.630000e+18,Twitter for Android,#lowerfoodprices \n\nSome people are eating bl...,OleNucho,254,False,0,0,NaN
1,2023-02-21 19:53:24+00:00,1.630000e+18,Twitter for Android,Now you understand that #LowerFoodPrices was a...,This_is_Kibande,"Kisumu, Kenya",False,1,0,NaN
2,2023-02-21 17:50:00+00:00,1.630000e+18,Twitter for Android,@omwambaKE I support this #lowerfoodprices,TracyspiceB,NaN,False,0,0,NaN
3,2023-02-21 12:35:38+00:00,1.630000e+18,Twitter for Android,Skyrocketing #HighCostOfLiving is a PUSH FACTO...,BazengaaKE,"Nairobi, Majengo.",False,7,3,NaN
4,2023-02-21 09:25:25+00:00,1.630000e+18,Twitter for Android,@OleItumbi last year a time like this you were...,alphoncetiffz,"Mombasa, Kenya",False,0,0,NaN


In [5]:
df1.drop(['ID', 'Place'], axis=1)

,Date,Source,Content,Username,Location,Verified,Likes,Retweets
0,2023-02-22 05:09:32+00:00,Twitter for Android,#lowerfoodprices \n\nSome people are eating bl...,OleNucho,254,False,0,0
1,2023-02-21 19:53:24+00:00,Twitter for Android,Now you understand that #LowerFoodPrices was a...,This_is_Kibande,"Kisumu, Kenya",False,1,0
2,2023-02-21 17:50:00+00:00,Twitter for Android,@omwambaKE I support this #lowerfoodprices,TracyspiceB,NaN,False,0,0
3,2023-02-21 12:35:38+00:00,Twitter for Android,Skyrocketing #HighCostOfLiving is a PUSH FACTO...,BazengaaKE,"Nairobi, Majengo.",False,7,3
4,2023-02-21 09:25:25+00:00,Twitter for Android,@OleItumbi last year a time like this you were...,alphoncetiffz,"Mombasa, Kenya",False,0,0
...,...,...,...,...,...,...,...,...
9996,2022-02-21 12:15:31+00:00,Twitter for Android,#lowerfoodprices \nhttps://t.co/Abl11N0qAa,IanMutethia,"Nairobi,kenya.",False,0,0
9997,2022-02-21 12:15:27+00:00,Twitter for Android,Weekend plan!!#TwendeCoast\n#lowerfoodprices #...,East80adventure,NaN,False,0,0
9998,2022-02-21 12:15:16+00:00,Twitter for Android,#LowerFoodPrices\n\nFood prices should actuall...,GitauWarigi,NaN,False,0,0
9999,2022-02-21 12:15:13+00:00,Twitter for Android,#lowerfoodprices\nClearly seems like a politic...,IregiKarue,NaN,False,0,0


In [6]:
df2 = pd.read_csv("Fuel.csv")
df2.head()

,Date,ID,Source,Content,Username,Location,Verified,Likes,Retweets,Place
0,2023-02-21 21:01:50+00:00,1.630000e+18,Twitter for iPhone,"In the land of #nofuel, &amp; c\ntime of #noca...",Oba_Temitope1,NaN,False,0,0,NaN
1,2023-02-19 06:25:07+00:00,1.630000e+18,Twitter for Android,"This time next week, we'll have a President-el...",Dr_GbeTV,"Abuja, Nigeria",False,2,1,NaN
2,2023-02-17 19:44:34+00:00,1.630000e+18,Twitter for iPhone,This Human is about to realize that feelings i...,smokejumper61,Washington State,False,0,0,NaN
3,2023-02-17 08:49:07+00:00,1.630000e+18,Twitter for iPhone,God should send @D_DeGea to Nigeria 🇳🇬 to save...,agarnacho7__,"Manchester, England UK",False,3,0,NaN
4,2023-02-16 10:34:51+00:00,1.630000e+18,Twitter for Android,Only February in this country 😂\n#nocash \n#no...,Daniel21760619,NaN,False,0,0,NaN


In [7]:
df2.drop(['ID', 'Place'], axis=1)

,Date,Source,Content,Username,Location,Verified,Likes,Retweets
0,2023-02-21 21:01:50+00:00,Twitter for iPhone,"In the land of #nofuel, &amp; c\ntime of #noca...",Oba_Temitope1,NaN,False,0,0
1,2023-02-19 06:25:07+00:00,Twitter for Android,"This time next week, we'll have a President-el...",Dr_GbeTV,"Abuja, Nigeria",False,2,1
2,2023-02-17 19:44:34+00:00,Twitter for iPhone,This Human is about to realize that feelings i...,smokejumper61,Washington State,False,0,0
3,2023-02-17 08:49:07+00:00,Twitter for iPhone,God should send @D_DeGea to Nigeria 🇳🇬 to save...,agarnacho7__,"Manchester, England UK",False,3,0
4,2023-02-16 10:34:51+00:00,Twitter for Android,Only February in this country 😂\n#nocash \n#no...,Daniel21760619,NaN,False,0,0
...,...,...,...,...,...,...,...,...
7878,2010-03-10 12:13:27+00:00,TweetDeck,#nolight #nofuel #nomoney #nojobs #nopeace #no...,Nwankpa_A,Abuja,True,0,0
7879,2010-03-08 21:14:25+00:00,Twitter Web Client,#Nigeria #Jos #Violence #ReligiousCrisis #Nige...,OluseyiAK,"Gusto Entertainment Hub, Lagos",False,0,0
7880,2010-03-02 02:37:01+00:00,Twitter Web Client,What's a hater? LOL #noFuel RT @TheDevilsCash:...,howardrjean,Global,False,0,0
7881,2009-06-24 12:05:38+00:00,Twitter Web Client,Não consigo pensar.... falta cafeína.RT @lgber...,henriqueeger,Joinville,False,0,0


Removing Duplicates

In [8]:
df.drop_duplicates(inplace=True)  # remove duplicate
df1.drop_duplicates(inplace=True)  # remove duplicate
df2.drop_duplicates(inplace=True)  # remove duplicate

Preprocessing: The date columns preesents both the date and the time frame, we remove the time part

In [9]:
# slicing the date , and removing the time portion

df['Date'] = df.Date.str.slice(0, 10)
print(df["Date"].unique())

df1['Date'] = df1.Date.str.slice(0, 10)
print(df1["Date"].unique())

df2['Date'] = df2.Date.str.slice(0, 10)
print(df2["Date"].unique())

['2023-02-22' '2023-02-21' '2023-02-20' '2023-02-19' '2023-02-18'
 '2023-02-17' '2023-02-16' '2023-02-15' '2023-02-14' '2023-02-13'
 '2023-02-12' '2023-02-11' '2023-02-10' '2023-02-09' '2023-02-08'
 '2023-02-07' '2023-02-06' '2023-02-05']
['2023-02-22' '2023-02-21' '2023-02-13' '2023-02-09' '2023-02-08'
 '2023-02-07' '2023-01-18' '2023-01-14' '2023-01-04' '2022-12-07'
 '2022-12-06' '2022-11-19' '2022-11-18' '2022-11-15' '2022-11-14'
 '2022-11-13' '2022-11-08' '2022-11-03' '2022-11-02' '2022-10-31'
 '2022-10-30' '2022-10-29' '2022-10-28' '2022-10-25' '2022-10-13'
 '2022-10-12' '2022-10-11' '2022-10-10' '2022-10-06' '2022-10-05'
 '2022-10-04' '2022-10-01' '2022-09-29' '2022-09-28' '2022-09-25'
 '2022-09-24' '2022-09-23' '2022-09-22' '2022-09-21' '2022-09-20'
 '2022-09-18' '2022-09-17' '2022-09-16' '2022-09-15' '2022-09-14'
 '2022-09-13' '2022-09-12' '2022-09-09' '2022-09-05' '2022-08-31'
 '2022-08-29' '2022-08-22' '2022-08-15' '2022-08-14' '2022-08-13'
 '2022-08-11' '2022-08-07' '2022-08

Analyzing the different languages used in the tweets, and narrowing down to English

In [10]:
from langdetect import detect_langs
from langdetect import detect
from langdetect import lang_detect_exception
from langdetect import DetectorFactory
DetectorFactory.seed = 0

In [11]:
# checking how many unique language
# tweets are present in the dataset
def detect_my(text):
    try:
        return detect(text)
    except:
        return 'Unknown'

In [12]:
df['language'] = df ['Content'].apply(detect_my)

In [13]:
df.head()

,Date,ID,Source,Content,Username,Location,Verified,Likes,Retweets,Place,language
0,2023-02-22,1.628290e+18,Twitter for Android,#ZelenskyWarCriminal #RussiaUkraineWar,manuel_moschini,"Florence, Italy.",False,0,0,NaN,fi
1,2023-02-22,1.628290e+18,Twitter for Android,#Russia 🇷🇺 / #Ukraine 🇺🇦\n\n‼️Air raid sirens ...,_Nex3_,Italia,False,0,0,NaN,en
2,2023-02-22,1.628290e+18,Twitter Web App,"নিষেধাজ্ঞার অধীনে থাকা জাহাজ প্রত্যাখ্যান, বাং...",tbsnewsdotnet,"Dhaka, Bangladesh",False,0,0,NaN,bn
3,2023-02-22,1.628290e+18,Twitter Web App,West media are not talking about President #Pu...,segunajayiemma,"Lagos,Nigeria",False,1,0,NaN,en
4,2023-02-22,1.628290e+18,Twitter for iPhone,#Russia #Ukraine #RussiaUkraineWar President D...,FalcDanny,NaN,False,0,0,NaN,en


# Extract Tweet Data

In [18]:
scraper = sntwitter.TwitterSearchScraper("#PoliceBrutality")

tweets = []

for i, tweet in enumerate (scraper.get_items()):
    df = [
        tweet.date,
        tweet.id, 
        tweet.content, 
        tweet.user.username, 
        tweet.likecount,
        tweet.retweetCount
    ]
    tweets.append(df)
    if i > 50:
        break

C:\Users\Getrude\AppData\Local\Temp\ipykernel_9068\462379057.py:9: FutureWarning: content is deprecated, use rawContent instead
  tweet.content,


AttributeError: 'Tweet' object has no attribute 'likecount'